# ECMWF Hackaton 

## Prerequisites
### DestinE Platform Credentials

You need to have an account on the [Destination Earth Platform](https://auth.destine.eu/realms/desp/account).

#### ⚠️ Warning: Authorized Access Only
The usage of this notebook and data access is reserved only to authorized user groups.

In [1]:
%%capture cap
%run ./cacheb-authentication.py

Username:  gaia.cipolletta
Password:  ········


In [2]:
output_1 = cap.stdout.split('}\n')
token = output_1[-1][0:-1]

from pathlib import Path
with open(Path.home() / ".netrc", "a") as fp:
    fp.write(token)

**NOTE**: the temporary password is valid for a limited period of time and needs to be regenerated and reconfigured periodically by running the cells above.

## Access/Read the Data
With a DESP account you can access the dataset in the CacheB cache with Xarray / Zarr.

Here below the list of available datasets, they are all reachable via a dedicated URL. In the following example we will work only with the Climate DT high resolution surface with hourly resolution, on atmosphere variables.

In [3]:
import xarray as xr

data = xr.open_dataset(
    "https://cacheb.dcms.destine.eu/d1-climate-dt/ScenarioMIP-SSP3-7.0-IFS-NEMO-0001-high-sfc-v0.zarr",
    storage_options={"client_kwargs":{"trust_env":"true"}},
    chunks={},
    engine="zarr",
)

In [4]:
data

<xarray.Dataset> Size: 800TB
Dimensions:           (time: 175320, latitude: 4096, longitude: 8193)
Coordinates:
    entireAtmosphere  float64 8B ...
  * latitude          (latitude) float64 33kB -90.0 -89.96 -89.91 ... 89.96 90.0
  * longitude         (longitude) float64 66kB -180.0 -180.0 ... 180.0 180.0
    step              timedelta64[ns] 8B ...
    surface           float64 8B ...
  * time              (time) datetime64[ns] 1MB 2020-01-01 ... 2039-12-31T23:...
Data variables: (12/34)
    blh               (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    chnk              (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    d2m               (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    e                 (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    ewss              (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    hcc               (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    ...                ...
    tp                (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    tprate            (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    tsr               (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    ttr               (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    u10               (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
    v10               (time, latitude, longitude) float32 24TB dask.array<chunksize=(48, 512, 512), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          1003
    history:                 2024-11-14T08:10 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

## Compliance with monitoring, log entry provisioning for the event `read_data`

Try to login the user_id starting from the token

Log an event recorded by the Service Provider in a single log entry following a general extendible
JSON format as follows:
```{
"@timestamp": “yyyy-MM-ddTHH:mm:ss.SSSZ”,
“service_name”: “value”,
"event_timestamp":" yyyy-MM-ddTHH:mm:ss.SSSZ ",
“event_type”: “value”,
“user_ID”:”value”
}
```

Where:
- @timestamp (string, required) is the timestamp indicating when the log entry was generated by
the service, formatted in ISO 8601 (yyyy-MM-ddTHH:mm:ss.SSSZ)

- service_name (string, required) is the name of the Service that is generating the log entry
- event_timestamp (string, required) is the timestamp of the event in ISO 8601 format (yyyy-MMddTHH:
mm:ss.SSSZ)
- event_type (string, required) is the name of the event, written with no spaces and no capital letters
or special characters (e.g. “resource_created” or “request_submitted”)
- user_ID (string, required) is the user ID as taken from the DestinE Platform Identity Management



### 1. Decode token (without verifying signature)
<details>
<summary>💡 Click here to show the solution</summary>

```python
import jwt  
import json
from datetime import datetime, timezone
import re 

match = re.search(r'password\s+([A-Za-z0-9\-\._]+)', token)
token = match.group(1).strip()
decoded = jwt.decode(token, options={"verify_signature": False})
print(decoded)
# Decode JWT without verifying signature
decoded = jwt.decode(token_str, options={"verify_signature": False})

# Extract and print the user_ID
user_id = decoded.get("sub")
print("User ID:", user_id)

### 2. Extract user_id
<details>
<summary>💡 Click here to show the solution</summary>

```python
user_id = decoded.get("sub") 
print("User ID:", user_id)


### Create the log entry in JSON format
<details>
<summary>💡 Click here to show the solution</summary>

```python
# Retrieve timestamp, create a mock service name and an event type
now_iso = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
print(now_iso)
service_name = 'onigiri'
event_type = 'read_dataset'

log_entry = {
    "@timestamp": now_iso,
    "service_name": service_name,
    "event_timestamp": now_iso,
    "event_type": event_type,
    "user_ID": user_id
}

print(json.dumps(log_entry, indent=4))

In [5]:
import jwt  
import json
from datetime import datetime, timezone
import re 

match = re.search(r'password\s+([A-Za-z0-9\-\._]+)', token)
token = match.group(1).strip()
decoded = jwt.decode(token, options={"verify_signature": False})
print(decoded)

{'iat': 1761931350, 'jti': 'af65be42-1bce-4401-ba0d-f122f866ab67', 'iss': 'https://auth.destine.eu/realms/desp', 'aud': 'https://auth.destine.eu/realms/desp', 'sub': '49649d01-585c-48dc-ab65-b3a3a0aaccfe', 'typ': 'Offline', 'azp': 'edh-public', 'sid': '724a3b9e-081a-4bb0-8cb0-60cb5ad18ce1', 'scope': 'openid offline_access basic'}


In [6]:
user_id = decoded.get("sub") 
print("User ID:", user_id)

User ID: 49649d01-585c-48dc-ab65-b3a3a0aaccfe


In [7]:
now_iso = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
print(now_iso)

2025-10-31T17:22:49.619Z


In [8]:
service_name = 'onigiri'
event_type = 'read_dataset'

In [9]:
log_entry = {
    "@timestamp": now_iso,
    "service_name": service_name,
    "event_timestamp": now_iso,
    "event_type": event_type,
    "user_ID": user_id
}

# --- Pretty print the log ---
print(json.dumps(log_entry, indent=4))

{
    "@timestamp": "2025-10-31T17:22:49.619Z",
    "service_name": "onigiri",
    "event_timestamp": "2025-10-31T17:22:49.619Z",
    "event_type": "read_dataset",
    "user_ID": "49649d01-585c-48dc-ab65-b3a3a0aaccfe"
}
